오차역전파법 <small>Backpropagation</small>
========
그래디언트를 컴퓨터로 Numerical하게 계산하려니 너무 시간이 오래걸린다. 미분을 열심히 해서, 해석적으로 구해보자!

In [1]:
import time
import numpy as np
import mnist
from common import sigmoid, gradient_sigmoid, softmax, cross_entropy_error_batch

#
# Hyper parameters
#
# 히든레이어 뉴런 수 (ex: 50, 100)
HIDDEN_LAYER_SIZE = 50
# 정규분포 난수로 생성될 초기 가중치의 표준편차
WEIGHT_INIT_STD = 0.01
# 학습에 사용할 미니배치의 크기
BATCH_SIZE = 100

#
# Main logic
#
MNIST = mnist.load()
TRAIN_IMG = MNIST['train_img']
TRAIN_LABEL = MNIST['train_label']

# 각 레이어의 뉴런 수
layer0_size = TRAIN_IMG.shape[-1]
layer1_size = HIDDEN_LAYER_SIZE
layer2_size = TRAIN_LABEL.shape[-1]

# 랜덤으로 웨이트 행렬과 가중치 벡터 초기화
parameters = [
    # w0
    WEIGHT_INIT_STD * np.random.randn(layer0_size, layer1_size), 
    # b0
    np.zeros(layer1_size),
    # w1
    WEIGHT_INIT_STD * np.random.randn(layer1_size, layer2_size),
    # b1
    np.zeros(layer2_size),
]

# 미니배치: 트레인셋에서 그래디언트 계산용으로 쓸 데이터 N개 임의추출
indices = np.random.choice(TRAIN_IMG.shape[0], BATCH_SIZE)
BATCH_IMG, BATCH_LABEL = TRAIN_IMG[indices], TRAIN_LABEL[indices]

def predict(w0, b0, w1, b1):
    a0 = BATCH_IMG @ w0 + b0
    z0 = sigmoid(a0)
    a1 = z0 @ w1 + b1
    z1 = softmax(a1)
    return [a0, z0, a1, z1]

def grad_analytic(parameters):
    w1 = parameters[2]
    a0, z0, _, expected = predict(*parameters)

    # Backward propagation
    dz1 = (expected - BATCH_LABEL)/BATCH_SIZE
    dw1 = z0.T @ dz1
    db1 = dz1.sum(axis=0)

    dz0 = gradient_sigmoid(a0) * (dz1 @ w1.T)
    dw0 = BATCH_IMG.T @ dz0
    db0 = dz0.sum(axis=0)
    return [dw0, db0, dw1, db1]

def grad_numerical(parameters, h=1E-4):
    def loss_function(*arguments):
        expected = predict(*arguments)[-1]
        return cross_entropy_error_batch(expected, BATCH_LABEL)
    def grad(param):
        shape = param.shape
        gradient = np.empty(shape)
        for j in np.ndindex(shape):
            orig = param[j]
            param[j] = orig + h
            y2 = loss_function(*parameters)
            param[j] = orig - h
            y1 = loss_function(*parameters)
            param[j] = orig
            gradient[j] = (y2 - y1)/(2*h)
        return gradient
    return [grad(param) for param in parameters]

print('\x1b[31m1. 계산시간 비교\x1b[0m')
# 해석적으로 그래디언트 계산
t0 = time.time()
analytic = grad_analytic(parameters)
t1 = time.time()
print(f'해석적으로 계산하는데에 든 시간 : {t1 - t0:.04}초')
# 수치적으로 그래디언트 계산
t0 = time.time()
numerical = grad_numerical(parameters)
t1 = time.time()
print(f'수치적으로 계산하는데에 든 시간 : {t1 - t0:.04}초\n\n')

# 행렬 출력
names = ['∇W0', '∇B0', '∇W1', '∇B1']
np.set_printoptions(suppress=True, precision=5, threshold=5)
print('\x1b[31m2. 그래디언트값 생김새\x1b[0m')
for name, mat in zip(names, analytic): print(f'  {name}\n{mat}\n')

# 해석적으로 계산한것과, 수치적으로 계산한것 비교
print('''
\x1b[31m3. 해석적으로 구한 그래디언트와, 수치적으로 구한 그래디언트가 얼마나 비슷할까?\x1b[0m
항목\t행렬 원소 이차평균\t수치적으로 계산한것과의 오차
------------------------------------------------------------''')
for analytic, numerical, name in zip(analytic, numerical, names):
    mean = np.sqrt((analytic**2).mean())
    error = np.sqrt(((analytic - numerical)**2).mean())

    print(f'{name}\t{mean:18.02}\t{error:28.02}')

1. 계산시간 비교
해석적으로 계산하는데에 든 시간 : 0.01059초
수치적으로 계산하는데에 든 시간 : 114.0초


2. 그래디언트값 생김새
  ∇W0
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]

  ∇B0
[-0.00001  0.00018 -0.00063 ...,  0.00001  0.00008  0.00025]

  ∇W1
[[-0.01316 -0.01855  0.01302 ...,  0.00942  0.01075 -0.02383]
 [-0.01078 -0.02     0.0136  ...,  0.01091  0.00979 -0.02437]
 [-0.01142 -0.02224  0.01473 ...,  0.01124  0.01004 -0.02444]
 ..., 
 [-0.00793 -0.02075  0.01371 ...,  0.00996  0.00993 -0.02591]
 [-0.00952 -0.02149  0.01331 ...,  0.01265  0.0091  -0.02555]
 [-0.01189 -0.02007  0.0152  ...,  0.01037  0.00919 -0.0253 ]]

  ∇B1
[-0.0237  -0.03996  0.02764 ...,  0.02184  0.01955 -0.04861]


3. 해석적으로 구한 그래디언트와, 수치적으로 구한 그래디언트가 얼마나 비슷할까?
항목	행렬 원소 이차평균	수치적으로 계산한것과의 오차
------------------------------------------------------------
∇W0	           0.00013	                     1.3e-